In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import glob
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows', 300)  # Show all rows


# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Updated_Target_Distance_Based_Data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['path',
 'NQ 09-24_Minute_60.csv',
 'ZB 09-24_Minute_60.csv',
 'ES 09-24_Minute_60.csv']

In [ ]:
def calculate_ideal_values(df):

    # Initialize lists to store ideal_buyer and ideal_seller values
    ideal_buyers = []
    ideal_sellers = []

    # Iterate over the rows of the DataFrame
    for i, row in df.iterrows():
        # Initialize the default values for ideal_buyer and ideal_seller
        ideal_buyer = 0
        ideal_seller = 0

        # Process ideal_buyer if BuyersInControlPrice is not zero
        if row['BuyersInControlPrice'] != 0:
            # Identify future rows
            future_rows = df.iloc[i:]
            # Find future matching rows where Close equals the current Stop value
            future_matching_rows = future_rows[future_rows['Close'] == row['Stop']]

            # If there are matching rows, find the maximum Close value up to the first matching row
            if not future_matching_rows.empty:
                max_future_close = future_rows.loc[:future_matching_rows.index[0], 'Close'].max()
                ideal_buyer = max_future_close

        # Process ideal_seller if SellersInControlPrice is not zero
        if row['SellersInControlPrice'] != 0:
            # Identify future rows
            future_rows = df.iloc[i:]
            # Find future matching rows where Close equals the current Stop value
            future_matching_rows = future_rows[future_rows['Close'] == row['Stop']]

            # If there are matching rows, find the minimum Close value up to the first matching row
            if not future_matching_rows.empty:
                min_future_close = future_rows.loc[:future_matching_rows.index[0], 'Close'].min()
                ideal_seller = min_future_close

        # Append the calculated values to the lists
        ideal_buyers.append(ideal_buyer)
        ideal_sellers.append(ideal_seller)

    # Add the new columns to the DataFrame
    df['ideal_buyer'] = ideal_buyers
    df['ideal_seller'] = ideal_sellers
    df['ideal_target'] = df['ideal_buyer'] + df['ideal_seller']

    # Optionally remove the 'ideal_buyer' and 'ideal_seller' columns if not needed
    df.drop(columns=['ideal_buyer', 'ideal_seller'], axis=1, inplace=True)

    return df

def process_files_in_directory(input_directory, output_directory):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Get all CSV files in the input directory
    files = [f for f in os.listdir(input_directory) if f.endswith('.csv')]

    for file in files:
        file_path = os.path.join(input_directory, file)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Drop the 'Unnamed: 0' column if it exists
        columns_to_drop = ['Unnamed: 0', 'ED_Target']
        df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

        # Convert 'Time Stamp' column to datetime if it exists
        if 'Time Stamp' in df.columns:
            df['Time Stamp'] = pd.to_datetime(df['Time Stamp'])

        # Apply the ideal values calculation function
        df = calculate_ideal_values(df)

        # Save the modified DataFrame to the output directory with a new file name
        output_file_name = f"Ideal_Target_{file}"
        output_file_path = os.path.join(output_directory, output_file_name)
        df.to_csv(output_file_path, index=False)

        print(f"Processed and saved {output_file_name} to {output_directory}")

In [ ]:
# Example usage
output_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Profit_Loss_Data/Profit_Loss_Real_Trader_data'
process_files_in_directory(data_folder, output_directory)

Processed and saved Ideal_Target_Updated_Target_ESDT_ES 09-24_Minute_60.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Ideal_Target_Data
Processed and saved Ideal_Target_Updated_Target_ESDT_NQ 09-24_Minute_60.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Ideal_Target_Data
Processed and saved Ideal_Target_Updated_Target_ESDT_ZB 09-24_Minute_60.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Ideal_Target_Data


In [ ]:
# df.drop(columns=["ED_Target"], inplace=True)